In [29]:
from ecmwf.opendata import Client as EcmwfClient
from client import Client 
import numpy as np

## ecmwf-opendata を使う

In [10]:
# 地上の風速の取得
ecmwf_client = EcmwfClient(source='aws')
result = ecmwf_client.retrieve(
    param=['10u', '10v'],
    target='data.grib2'
)
print(result.datetime)

2024-04-20 18:00:00


# pygrib を使う

In [25]:
# ecmwf のオープンデータに含まれるすべてのデータ項目を表示させる
import pygrib
grib_dir = "ecmwf/20240429/12z/ifs/0p25/oper"
grib_file = "20240429120000-3h-oper-fc.grib2" 

# GRIB ファイルを開く
with pygrib.open(f"{grib_dir}/{grib_file}") as grbs:
    # ファイル内の全レコードを反復処理する
    print(short_name in ['d', 'gh', 'q', 'r', 't', 'u', 'v', 'w', 'vo'])
    
    for n, grb in enumerate(grbs):
        single_or_pressure = 'pressure' if (short_name := grb.shortName) in ['d', 'gh', 'q', 'r', 't', 'u', 'v', 'w', 'vo'] else 'single'    
        print(f"{n}: {single_or_pressure}: {short_name}: {grb.name}: {grb.level} {grb.typeOfLevel}")

True
0: pressure: v: V component of wind: 925 isobaricInhPa
1: pressure: u: U component of wind: 925 isobaricInhPa
2: pressure: u: U component of wind: 250 isobaricInhPa
3: pressure: v: V component of wind: 250 isobaricInhPa
4: pressure: t: Temperature: 500 isobaricInhPa
5: pressure: u: U component of wind: 200 isobaricInhPa
6: pressure: v: V component of wind: 200 isobaricInhPa
7: pressure: gh: Geopotential height: 700 isobaricInhPa
8: pressure: r: Relative humidity: 850 isobaricInhPa
9: pressure: gh: Geopotential height: 850 isobaricInhPa
10: pressure: gh: Geopotential height: 1000 isobaricInhPa
11: pressure: v: V component of wind: 600 isobaricInhPa
12: pressure: u: U component of wind: 600 isobaricInhPa
13: pressure: r: Relative humidity: 700 isobaricInhPa
14: pressure: v: V component of wind: 500 isobaricInhPa
15: pressure: u: U component of wind: 500 isobaricInhPa
16: single: 100v: 100 metre V wind component: 100 heightAboveGround
17: single: 100u: 100 metre U wind component: 100

In [31]:
# 気温のデータを表示させる
import pygrib
grib_dir = "ecmwf/20240429/12z/ifs/0p25/oper"
grib_file = "20240429120000-3h-oper-fc.grib2" 

# GRIB ファイルを開く
with pygrib.open(f"{grib_dir}/{grib_file}") as grbs:
    selected_grb = grbs.select(shortName='2t')[0]
    print(f"Parameter Name: {grb.name}")
    print(f"Valid Time: {grb.validDate}")
    print(f"Data Type: {grb.dataType}")
    print(f"Values: {grb.values.shape}")
    #データを表示
    data = grb.values
    print(type(data))
    print(data)

Parameter Name: Vorticity (relative)
Valid Time: 2024-04-29 15:00:00
Data Type: fc
Values: (721, 1440)
<class 'numpy.ndarray'>
[[-3.87069304e-05 -3.87069304e-05 -3.87069304e-05 ... -3.87069304e-05
  -3.87069304e-05 -3.87069304e-05]
 [-3.73658258e-05 -3.74254305e-05 -3.74850351e-05 ... -3.72168142e-05
  -3.72764189e-05 -3.73062212e-05]
 [-3.07795126e-05 -3.08987219e-05 -3.10179312e-05 ... -3.04516871e-05
  -3.05708963e-05 -3.06901056e-05]
 ...
 [-4.97039873e-05 -4.97933943e-05 -4.98828012e-05 ... -4.94357664e-05
  -4.95251734e-05 -4.96145803e-05]
 [-5.24756033e-05 -5.24756033e-05 -5.24458010e-05 ... -5.24756033e-05
  -5.24756033e-05 -5.24756033e-05]
 [-4.90483362e-05 -4.90483362e-05 -4.90483362e-05 ... -4.90483362e-05
  -4.90483362e-05 -4.90483362e-05]]


# ecmwf-weather を使う

In [11]:
# ecmwfのオープンデータの最新のものをダウンロード
client = Client()
client.download(steps=[0, 3, 6, 9,12], target_folder='ecmwf')

200

In [1]:
# gribデータから指定するsingleのデータ項目の取得

from ecmwf_data import ECMWFData

grib_dir = "ecmwf/20240429/06z/ifs/0p25/scda"
grib_file = "20240429060000-0h-scda-fc.grib2" # GRIBファイルのパス

data = ECMWFData(file_path=f"{grib_dir}/{grib_file}")
u = data.extract_single(short_name='10u')
print(u)
print(type(u))
print(u.values)


1:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 0 hrs:from 202404290600
<class 'pygrib._pygrib.gribmessage'>
[[ 1.08352661  1.08352661  1.08352661 ...  1.08352661  1.08352661
   1.08352661]
 [ 1.42727661  1.42727661  1.42727661 ...  1.42727661  1.42727661
   1.42727661]
 [ 1.61477661  1.63040161  1.63040161 ...  1.59915161  1.61477661
   1.61477661]
 ...
 [-2.15084839 -2.16647339 -2.18209839 ... -2.10397339 -2.11959839
  -2.13522339]
 [-1.16647339 -1.18209839 -1.21334839 ... -1.08834839 -1.10397339
  -1.13522339]
 [-7.60397339 -7.60397339 -7.60397339 ... -7.60397339 -7.60397339
  -7.60397339]]


In [3]:
# gribデータから指定するデータ項目及びpressureの取得
from ecmwf_data import ECMWFData

grib_dir = "ecmwf/20240429/06z/ifs/0p25/scda"
grib_file = "20240429060000-0h-scda-fc.grib2" # GRIBファイルのパス

data = ECMWFData(file_path=f"{grib_dir}/{grib_file}")
u = data.extract_pressure(short_name='u', level='850')
print(u)
print(type(u))
print(u.values)



1:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 85000 Pa:fcst time 0 hrs:from 202404290600
<class 'pygrib._pygrib.gribmessage'>
[[ 1.93275452  1.93275452  1.93275452 ...  1.93275452  1.93275452
   1.93275452]
 [-1.37974548 -1.37974548 -1.37974548 ... -1.37974548 -1.37974548
  -1.37974548]
 [-1.28599548 -1.28599548 -1.28599548 ... -1.25474548 -1.28599548
  -1.28599548]
 ...
 [-2.00474548 -2.03599548 -2.06724548 ... -1.97349548 -1.97349548
  -2.00474548]
 [-1.12974548 -1.16099548 -1.16099548 ... -1.03599548 -1.06724548
  -1.09849548]
 [-3.66099548 -3.66099548 -3.66099548 ... -3.66099548 -3.66099548
  -3.66099548]]


In [5]:
 # gribデータから指定するデータ項目及びpressureの取得
from ecmwf_data import ECMWFData

grib_dir = "ecmwf/20240429/06z/ifs/0p25/scda"
grib_file = "20240429060000-0h-scda-fc.grib2" # GRIBファイルのパス

data = ECMWFData(file_path=f"{grib_dir}/{grib_file}")
u = data.extract_pressure(short_name='u', level='850')

# grb のすべての属性とその値を表示
for attr in dir(u):
    # __から始まる特殊メソッド/属性は除外
    if not attr.startswith('__'):
        try:
            value = getattr(u, attr)
            if callable(value):
                continue  # 呼び出し可能なメソッドは除外
            print(f"{attr}: {value}")
        except Exception as e:
            print(f"{attr}: この属性を読み込めません - {e}")


AEC_DATA_3BYTE_OPTION_MASK: 1
AEC_DATA_MSB_OPTION_MASK: 1
AEC_DATA_PREPROCESS_OPTION_MASK: 1
AEC_DATA_SIGNED_OPTION_MASK: 0
AEC_PAD_RSI_OPTION_MASK: 0
AEC_RESTRICTED_OPTION_MASK: 0
GRIBEditionNumber: 2
NV: 0
Ni: 1440
Nj: 721
PLPresent: 0
PVPresent: 0
_all_keys: ['globalDomain', 'GRIBEditionNumber', 'tablesVersionLatestOfficial', 'tablesVersionLatest', 'grib2divider', 'angleSubdivisions', 'missingValue', 'ieeeFloats', 'isHindcast', 'section0Length', 'identifier', 'discipline', 'editionNumber', 'totalLength', 'sectionNumber', 'section1Length', 'numberOfSection', 'centre', 'centreDescription', 'subCentre', 'tablesVersion', 'masterDir', 'localTablesVersion', 'significanceOfReferenceTime', 'year', 'month', 'day', 'hour', 'minute', 'second', 'dataDate', 'julianDay', 'dataTime', 'productionStatusOfProcessedData', 'typeOfProcessedData', 'md5Section1', 'selectStepTemplateInterval', 'selectStepTemplateInstant', 'stepType', 'is_chemical', 'is_chemical_distfn', 'is_chemical_srcsink', 'is_aerosol',

In [31]:
import json
from ecmwf_data import ECMWFData


grib_dir = "ecmwf/20240429/06z/ifs/0p25/scda"
grib_file = "20240429060000-0h-scda-fc.grib2" # GRIBファイルのパス
data = ECMWFData(file_path=f"{grib_dir}/{grib_file}")
u = data.extract_single('10u')
# print(u.values.flatten().tolist())
u_list = np.round(u.values.flatten(), 1).tolist()
json_string = json.dumps(u_list)
print(json_string[:10000])

[1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1,